<a href="https://colab.research.google.com/github/rupaidutta66/PySpark/blob/main/BundesligaAnalysis_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step 1: Import necessary libraries and set up Spark in Google Colab**

In [29]:

!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, row_number
from pyspark.sql.window import Window
import pandas as pd




# Step 2: Load the provided datasets into Spark DataFrames

In [30]:
# Create a Spark session
spark = SparkSession.builder.appName("BundesligaAnalysis").getOrCreate()


# Load match.csv
match_df = spark.read.csv("/content/Matches.csv", header=True, inferSchema=True)

# Load teams_in_matches.csv
teams_in_matches_df = spark.read.csv("/content/Teams_in_Matches.csv", header=True, inferSchema=True)

# Load teams.csv
teams_df = spark.read.csv("/content/Teams.csv", header=True, inferSchema=True)

# Load unique_teams.csv
unique_teams_df = spark.read.csv("/content/Unique_Teams.csv", header=True, inferSchema=True)



# Step 3: Perform data transformations and calculations to answer the questions

### 1. Who are the winners of the D1 division in the Germany Football Association (Bundesliga) in the last decade?

In [ ]:


# Filter for D1 division and matches from the last decade (2007-2017)
last_decade_winners = match_df.filter((match_df["Div"] == "D1") & (year(match_df["Date"]) >= 2007) & (year(match_df["Date"]) <= 2017))

# Find the teams that won each match
last_decade_winners = last_decade_winners.withColumn("Winner", (col("FTR") == "H").cast("int"))

# Group by season and team to count the number of wins
winners_by_season = last_decade_winners.groupBy("Season", "HomeTeam").agg({"Winner": "sum"}).withColumnRenamed("sum(Winner)", "Wins")

# Find the team with the most wins in each season
window_spec = Window.partitionBy("Season").orderBy(col("Wins").desc())
season_winners = winners_by_season.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1)

# Join with unique_teams_df to get team names
season_winners = season_winners.join(unique_teams_df, winners_by_season["HomeTeam"] == unique_teams_df["TeamName"]).select("Season", "TeamName")

# Show the winners of the D1 division in the last decade
print("Winners of the D1 division in the last decade:")
season_winners.show()


### 2. Which teams have been relegated in the past 10 years?

In [ ]:


# Filter for D1 division and matches from the last decade (2007-2017)
last_10_years_matches = match_df.filter((year(match_df["Date"]) >= 2007) & (year(match_df["Date"]) <= 2017))

# Group by season and team, and calculate their final position in the division
relegation_table = last_10_years_matches.filter(last_10_years_matches["Div"] == "D1").groupBy("Season", "HomeTeam").agg({"FTHG": "sum", "FTAG": "sum"})

# Assuming that the bottom 3 teams in the division get relegated, you can calculate their positions
relegation_table = relegation_table.withColumn("FinalPosition", relegation_table["sum(FTHG)"] - relegation_table["sum(FTAG)"])

# Filter for teams that finished in the relegation zone (bottom 3)
relegated_teams = relegation_table.filter(relegation_table["FinalPosition"] > 0).orderBy("Season", "FinalPosition")

# Join with unique_teams_df to get team names
relegated_teams = relegated_teams.join(unique_teams_df, relegation_table["HomeTeam"] == unique_teams_df["TeamName"]).select("Season", "TeamName")

# Show the teams that were relegated in the past 10 years
print("Teams relegated in the past 10 years:")
relegated_teams.show()



###  3. Does Octoberfest affect the performance of Bundesliga?

In [33]:

# Calculate the average goals scored in October vs. other months
october_goals = match_df.filter(month(match_df["Date"]) == 10)
other_months_goals = match_df.filter(month(match_df["Date"]) != 10)

# Calculate the average goals scored in each group
october_avg_goals = october_goals.selectExpr("avg(FTHG) as OctoberAvgHomeGoals", "avg(FTAG) as OctoberAvgAwayGoals")
other_months_avg_goals = other_months_goals.selectExpr("avg(FTHG) as OtherMonthsAvgHomeGoals", "avg(FTAG) as OtherMonthsAvgAwayGoals")

# Show the average goals scored in October vs. other months
print("Average goals scored in October vs. other months:")
october_avg_goals.crossJoin(other_months_avg_goals).show()

Average goals scored in October vs. other months:
+-------------------+-------------------+-----------------------+-----------------------+
|OctoberAvgHomeGoals|OctoberAvgAwayGoals|OtherMonthsAvgHomeGoals|OtherMonthsAvgAwayGoals|
+-------------------+-------------------+-----------------------+-----------------------+
| 1.5704169944925255| 1.1066089693154997|     1.5108907304261197|     1.0964542861024318|
+-------------------+-------------------+-----------------------+-----------------------+



### 4. Which season of Bundesliga was the most competitive in the last decade?

In [ ]:

# Calculate the standard deviation of goal differences for each season in the last decade
goal_difference = match_df.filter(year(match_df["Date"]) >= 2006).withColumn("GoalDifference", col("FTHG") - col("FTAG"))
std_dev_by_season = goal_difference.groupBy("Season").agg({"GoalDifference": "stddev_pop"}).withColumnRenamed("stddev_pop(GoalDifference)", "StdDevGoalDifference")

# Find the season with the highest standard deviation
most_competitive_season = std_dev_by_season.orderBy(col("StdDevGoalDifference").desc()).first()

# Show the most competitive season
print("The most competitive season in the last decade was:", most_competitive_season["Season"])

### 5. What's the best month to watch Bundesliga?

In [35]:

# Calculate the average goals scored in each month
monthly_avg_goals = match_df.groupBy(month(match_df["Date"]).alias("Month")).agg({"FTHG": "avg", "FTAG": "avg"}).orderBy("Month")

# Show the average goals scored in each month
print("Average goals scored in each month:")
monthly_avg_goals.show()

Average goals scored in each month:
+-----+------------------+------------------+
|Month|         avg(FTAG)|         avg(FTHG)|
+-----+------------------+------------------+
|    1|1.0217881292261457|1.4282494365138994|
|    2| 1.068510111432109|1.4424267437061493|
|    3|1.0603953147877012|1.5183016105417277|
|    4|  1.06371790906279|1.4825239715434582|
|    5|1.2057831325301205| 1.687710843373494|
|    6|          1.453125|         1.9765625|
|    7|0.7570093457943925|1.1308411214953271|
|    8|1.1429832303618712|1.4717563989408649|
|    9|1.1246044303797469|1.5395569620253164|
|   10|1.1066089693154997|1.5704169944925255|
|   11|  1.08994708994709|1.5366591080876795|
|   12|1.0872226472838562|1.4762815608263198|
+-----+------------------+------------------+



# THANK YOU FOR WATCHING